In [13]:
import sys
sys.path.append('../unseen')

import fileio
import bias_correction
import time_utils
import array_handling

In [5]:
fcst_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_19900501-19931101_A-DEC-sum_cafe-grid-TAS-POINT.zarr.zip'
obs_file = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'
output_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_awap-additive-correction_19900501-19931101_A-DEC-sum_cafe-grid-TAS-POINT.zarr.zip'

In [6]:
ds_fcst = fileio.open_file(fcst_file)
ds_fcst

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 8, lead_time: 11)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (lead_time, init_date) object dask.array<chunksize=(11, 8), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble) float32 dask.array<chunksize=(1, 11, 96), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    history:    Mon Jul 05 17:10:11 2021: /g/data/e14/dbi599/miniconda3/envs/...
    title:      AccessOcean-AM2

In [7]:
da_fcst = ds_fcst['pr']
da_fcst

<xarray.DataArray 'pr' (init_date: 8, lead_time: 11, ensemble: 96)>
dask.array<open_dataset-fdb4e24967405200f3325f03c208d0bcpr, shape=(8, 11, 96), dtype=float32, chunksize=(1, 11, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (lead_time, init_date) object dask.array<chunksize=(11, 8), meta=np.ndarray>
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          mm d-1

In [10]:
init_dates = time_utils.cftime_to_str(da_fcst['init_date'])
init_dates

['1990-12-31',
 '1990-12-31',
 '1991-12-31',
 '1991-12-31',
 '1992-12-31',
 '1992-12-31',
 '1993-12-31',
 '1993-12-31']

In [11]:
n_lead_steps = int(da_fcst['lead_time'].values.max()) + 1
print(n_lead_steps)

11


In [12]:
ds_obs = fileio.open_file(obs_file, variables=['pr'])
ds_obs

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) object 1900-12-31 00:00:00 ... 2020-12-31 00:00:00
Data variables:
    pr       (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 13:54:07 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [14]:
ds_obs = array_handling.stack_by_init_date(ds_obs, init_dates, n_lead_steps)
ds_obs

<xarray.Dataset>
Dimensions:    (init_date: 8, lead_time: 11)
Coordinates:
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (init_date, lead_time) object 1990-12-31 00:00:00 ... 2003-12-...
Data variables:
    pr         (init_date, lead_time) float64 dask.array<chunksize=(8, 11), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 13:54:07 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [15]:
da_obs = ds_obs['pr']
da_obs

<xarray.DataArray 'pr' (init_date: 8, lead_time: 11)>
dask.array<getitem, shape=(8, 11), dtype=float64, chunksize=(8, 11), chunktype=numpy.ndarray>
Coordinates:
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (init_date, lead_time) object 1990-12-31 00:00:00 ... 2003-12-...
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm d-1
    valid_range:                   [-1.0, 100000.0]

In [16]:
bias = bias_correction.get_bias(da_fcst, da_obs, 'additive')
bias

<xarray.DataArray 'pr' (month: 1, lead_time: 11)>
dask.array<sub, shape=(1, 11), dtype=float64, chunksize=(1, 11), chunktype=numpy.ndarray>
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
  * month      (month) int64 12
Attributes:
    bias_correction_method:  additive

In [17]:
da_fcst_bc = bias_correction.remove_bias(da_fcst, bias, 'additive')
da_fcst_bc

<xarray.DataArray 'pr' (init_date: 8, lead_time: 11, ensemble: 96)>
dask.array<sub, shape=(8, 11, 96), dtype=float64, chunksize=(1, 11, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (lead_time, init_date) object dask.array<chunksize=(11, 8), meta=np.ndarray>
Attributes:
    bias_correction_method:  additive

In [19]:
ds_fcst_bc = da_fcst_bc.to_dataset()
infile_logs = {fcst_file: ds_fcst.attrs['history'],
               obs_file: ds_obs.attrs['history']}
ds_fcst_bc.attrs['history'] = fileio.get_new_log(infile_logs=infile_logs)
ds_fcst_bc

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 8, lead_time: 11)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (lead_time, init_date) object dask.array<chunksize=(11, 8), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble) float64 dask.array<chunksize=(1, 11, 96), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 14:12:35 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [20]:
ds_fcst_bc = ds_fcst_bc.chunk({'init_date': -1, 'lead_time': -1})
fileio.to_zarr(ds_fcst_bc, output_file)